In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split, LeaveOneOut, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error
df = pd.read_csv('IphoneData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
X = newdf.to_numpy()
y = df['CO2E'].to_numpy()

In [15]:
from sklearn.pipeline import Pipeline


xgBoost_SearchSpace = {
    'booster': ['gblinear'],
    'n_estimators': [225],
    'learning_rate': [.225],
    'reg_lambda': [0, 0.05, .1],
    'reg_alpha': [0, 0.05, .1],
    'eval_metric': ['rmse', 'auc'],
    'feature_selector': ['cyclic', 'random', 'greedy', 'thrifty'],
    'objective': ['reg:squaredlogerror'],
    'updater': ['shotgun', 'coord_descent'],
    'top_k': [0, 1 , 2, 3]            
}

rf_SearchSpace = {
    'criterion': ['mse', 'friedman_mse', 'mae'],  # Splitting criterion
    'splitter': ['best', 'random'],               # Strategy to choose split at each node
    'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],      # Maximum depth of the tree
    'min_samples_split': [2, 5, 10, 15, 20],              # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4, 8, 16],                # Minimum number of samples required to be at a leaf node
    'max_features': ['auto', 'sqrt', 'log2', None], # Number of features to consider for the best split
    'max_leaf_nodes': [None, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100]        # Maximum number of leaf nodes in the tree
}

svr_SearchSpace = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto', 0.1, 1],
    'epsilon': [0.1, 0.2, 0.5]
}

lasso_SearchSpace = {
    'alpha': [.1, .125, .175, .2, .15, .18, .17, .165, .185],  # Different values for alpha
    'tol': [.0001],  # Different values for tolerance
    'max_iter': [2000],  # Different values for maximum iterations
    'selection': ['cyclic']
}


scoring = {
    'r2': 'r2',  
    'neg_mean_squared_error': 'neg_mean_squared_error'
}

"""
lbfgs search
    'hidden_layer_sizes': [(10, 20, 10)],  # Adjusting number of neurons and layers
    'activation': ['identity'],  # Activation functions 
    'solver': ['lbfgs'],  # Optimization algorithms
    'batch_size': [25],  # Batch size
    'max_iter': [2500],  # Maximum number of iterations
    'random_state': [42],

    'tol': [1e-4, 1e-5, 1e-6, 1e-3, 1e-2, 1e-1],
    'alpha': [0.001],
    'early_stopping': [True, False],
    'validation_fraction': [],
    'max_fun': []
"""

searchSpace = {
    'hidden_layer_sizes': [(10, 10, 10, 10)],  # Adjusting number of neurons and layers
    'activation': ['identity'],  # Activation functions 
    'solver': ['lbfgs'],  # Optimization algorithms
    'batch_size': [25],  # Batch size
    'max_iter': [5000],  # Maximum number of iterations
    'random_state': [42],

    'tol': [1e-4, 1e-5, 1e-6, 1e-3, 1e-2, 1e-1],
    'alpha': [0.001, 0.01, 0.1, 1],
    'early_stopping': [True, False],
    'validation_fraction': [0.1, 0.001, 0.0001, 0.00001, 1],
    'max_fun': [150000, 12000, 10000, 17500, 20000]
}


In [16]:


GS = GridSearchCV(estimator=MLPRegressor(),
                  param_grid= searchSpace,
                  scoring = 'neg_mean_absolute_error',
                  refit='neg_mean_absolute_error',
                  cv=5,
                  verbose=4)

In [17]:
GS.fit(X, y)

Fitting 5 folds for each of 1200 candidates, totalling 6000 fits
[CV 1/5] END activation=identity, alpha=0.001, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.1;, score=-16.853 total time=   0.2s
[CV 2/5] END activation=identity, alpha=0.001, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.1;, score=-16.936 total time=   0.1s
[CV 3/5] END activation=identity, alpha=0.001, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.1;, score=-6.343 total time=   0.4s
[CV 4/5] END activation=identity, alpha=0.001, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lb

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 4/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.1;, score=-4.091 total time=   0.6s
[CV 5/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.1;, score=-5.347 total time=   0.5s
[CV 1/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.001;, score=-19.194 total time=   0.4s
[CV 2/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.001;, score=-13.675 total time=   0.0s
[C

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 4/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.001;, score=-4.091 total time=   0.6s
[CV 5/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.001;, score=-5.347 total time=   0.5s
[CV 1/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.0001;, score=-19.194 total time=   0.4s
[CV 2/5] END activation=identity, alpha=1, batch_size=25, early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10), max_fun=150000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.0001, validation_fraction=0.0001;, score=-13.675 total time=   0

...

[CV 4/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=0.0001;, score=-4.091 total time=   0.6s
[CV 5/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=0.0001;, score=-5.347 total time=   0.5s
[CV 1/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1e-05;, score=-19.194 total time=   0.4s
[CV 2/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1e-05;, score=-13.675 total time=   0.0s
[CV 3/5]

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:546: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


[CV 4/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1e-05;, score=-4.091 total time=   0.6s
[CV 5/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1e-05;, score=-5.347 total time=   0.5s
[CV 1/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1;, score=nan total time=   0.0s
[CV 2/5] END activation=identity, alpha=1, batch_size=25, early_stopping=False, hidden_layer_sizes=(10, 10, 10, 10), max_fun=20000, max_iter=5000, random_state=42, solver=lbfgs, tol=0.1, validation_fraction=1;, score=nan total time=   0.0s
[CV 3/5] END activation=id

C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py:542: FitFailedWarning: 
1200 fits failed out of a total of 6000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_validation.py", line 890, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\loca

GridSearchCV(cv=5, estimator=MLPRegressor(),
             param_grid={'activation': ['identity'],
                         'alpha': [0.001, 0.01, 0.1, 1], 'batch_size': [25],
                         'early_stopping': [True, False],
                         'hidden_layer_sizes': [(10, 10, 10, 10)],
                         'max_fun': [150000, 12000, 10000, 17500, 20000],
                         'max_iter': [5000], 'random_state': [42],
                         'solver': ['lbfgs'],
                         'tol': [0.0001, 1e-05, 1e-06, 0.001, 0.01, 0.1],
                         'validation_fraction': [0.1, 0.001, 0.0001, 1e-05, 1]},
             refit='neg_mean_absolute_error', scoring='neg_mean_absolute_error',
             verbose=4)

In [18]:
GS.best_params_

{'activation': 'identity',
 'alpha': 0.1,
 'batch_size': 25,
 'early_stopping': True,
 'hidden_layer_sizes': (10, 10, 10, 10),
 'max_fun': 150000,
 'max_iter': 5000,
 'random_state': 42,
 'solver': 'lbfgs',
 'tol': 0.1,
 'validation_fraction': 0.1}

In [19]:
GS.best_score_

-9.602119218049522

In [20]:
GS.best_estimator_

MLPRegressor(activation='identity', alpha=0.1, batch_size=25,
             early_stopping=True, hidden_layer_sizes=(10, 10, 10, 10),
             max_fun=150000, max_iter=5000, random_state=42, solver='lbfgs',
             tol=0.1)